In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset
data = pd.read_json('./drive/MyDrive/NN/DF_NN.json')
print('done-1')
# Convert x_values and y_values from string representation to lists
print('done-2')
# Find the maximum sequence length
max_len = max(data['x_values'].apply(len).max(), data['y_values'].apply(len).max())
print('done-3')
# Pad sequences to the same length
data['x_values_padded'] = pad_sequences(data['x_values'], maxlen=max_len, padding='post', dtype='float32').tolist()
data['y_values_padded'] = pad_sequences(data['y_values'], maxlen=max_len, padding='post', dtype='float32').tolist()
print('done-4')
# Combine x and y coordinates into a single array for each trial
data['coordinates'] = data.apply(lambda row: np.stack((row['x_values_padded'], row['y_values_padded']), axis=1), axis=1)
print('done-5')
# Prepare the features and labels
X = np.stack(data['coordinates'].values)
y = data['status'].values
print('done-6')
# Encode labels to binary values (0 or 1)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print('done-7')
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('done-8')

done-1
done-2
done-3
done-4
done-5
done-6
done-7
done-8


## Simple Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking

# Define the model
model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_len, 2)),  # Mask padding values
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
1329/1329 [==============================] - 44s 24ms/step - loss: 0.3545 - accuracy: 0.8477 - val_loss: 0.3151 - val_accuracy: 0.8733
Epoch 2/50
1329/1329 [==============================] - 30s 23ms/step - loss: 0.3042 - accuracy: 0.8735 - val_loss: 0.2891 - val_accuracy: 0.8806
Epoch 3/50
1329/1329 [==============================] - 30s 22ms/step - loss: 0.2798 - accuracy: 0.8859 - val_loss: 0.2750 - val_accuracy: 0.8897
Epoch 4/50
1329/1329 [==============================] - 30s 22ms/step - loss: 0.2598 - accuracy: 0.8946 - val_loss: 0.2592 - val_accuracy: 0.8962
Epoch 5/50
1329/1329 [==============================] - 29s 22ms/step - loss: 0.2493 - accuracy: 0.9012 - val_loss: 0.2616 - val_accuracy: 0.8943
Epoch 6/50
1329/1329 [==============================] - 30s 23ms/step - loss: 0.2439 - accuracy: 0.9029 - val_loss: 0.2412 - val_accuracy: 0.9052
Epoch 7/50
1329/1329 [==============================] - 30s 22ms/step - loss: 0.2356 - accuracy: 0.9067 - val_loss: 0.2430 -

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
# Load the model from the file
model.save('./drive/MyDrive/NN/NN_model.h5')


333/333 [==============================] - 2s 7ms/step - loss: 0.2196 - accuracy: 0.9170
Test Accuracy: 91.70%


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Advanced Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Define the model
model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_len, 2)),  # Mask padding values
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

# Define learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

# Compile the model with a custom learning rate schedule
model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test),
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])


Epoch 1/50
665/665 [==============================] - 45s 49ms/step - loss: 0.3427 - accuracy: 0.8559 - val_loss: 0.3096 - val_accuracy: 0.8791 - lr: 0.0010
Epoch 2/50
665/665 [==============================] - 27s 41ms/step - loss: 0.2800 - accuracy: 0.8858 - val_loss: 0.2874 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 3/50
665/665 [==============================] - 26s 40ms/step - loss: 0.2584 - accuracy: 0.8959 - val_loss: 0.2654 - val_accuracy: 0.8909 - lr: 0.0010
Epoch 4/50
665/665 [==============================] - 26s 40ms/step - loss: 0.2468 - accuracy: 0.9021 - val_loss: 0.2368 - val_accuracy: 0.9074 - lr: 0.0010
Epoch 5/50
665/665 [==============================] - 25s 38ms/step - loss: 0.2387 - accuracy: 0.9060 - val_loss: 0.2474 - val_accuracy: 0.9024 - lr: 0.0010
Epoch 6/50
665/665 [==============================] - 26s 39ms/step - loss: 0.2275 - accuracy: 0.9099 - val_loss: 0.2295 - val_accuracy: 0.9107 - lr: 0.0010
Epoch 7/50
665/665 [==============================] - 26s 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
# Load the model from the file
model.save('./drive/MyDrive/NN/NN_advanced_model.h5')

333/333 [==============================] - 4s 11ms/step - loss: 0.2017 - accuracy: 0.9212
Test Accuracy: 92.12%


## Power Intensive Model


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy

# Enable mixed precision training
set_global_policy('mixed_float16')

# Define the model within a strategy scope if using multiple GPUs
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = Sequential([
        Masking(mask_value=0.0, input_shape=(max_len, 2)),  # Mask padding values
        LSTM(256, return_sequences=True),  # Increase units
        Dropout(0.3),
        LSTM(128, return_sequences=True),
        Dropout(0.3),
        LSTM(64),
        Dense(1, activation='sigmoid', dtype='float32')  # Ensure output is float32
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks with correct filepath
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, save_weights_only=False)

# Train the model with a larger batch size
history = model.fit(X_train, y_train, epochs=50, batch_size=256, validation_data=(X_test, y_test),
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
# Load the model from the file
model.save('./drive/MyDrive/NN/NN_power_model.h5')

Epoch 1/50
167/167 [==============================] - 40s 117ms/step - loss: 0.3396 - accuracy: 0.8551 - val_loss: 0.2742 - val_accuracy: 0.8858 - lr: 0.0010
Epoch 2/50
  2/167 [..............................] - ETA: 8s - loss: 0.2384 - accuracy: 0.9004

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


167/167 [==============================] - 14s 81ms/step - loss: 0.2671 - accuracy: 0.8917 - val_loss: 0.2686 - val_accuracy: 0.8934 - lr: 0.0010
Epoch 3/50
167/167 [==============================] - 14s 85ms/step - loss: 0.2424 - accuracy: 0.9032 - val_loss: 0.2456 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 4/50
167/167 [==============================] - 15s 89ms/step - loss: 0.2322 - accuracy: 0.9075 - val_loss: 0.2319 - val_accuracy: 0.9088 - lr: 0.0010
Epoch 5/50
167/167 [==============================] - 15s 93ms/step - loss: 0.2279 - accuracy: 0.9097 - val_loss: 0.2365 - val_accuracy: 0.9092 - lr: 0.0010
Epoch 6/50
167/167 [==============================] - 15s 90ms/step - loss: 0.2177 - accuracy: 0.9149 - val_loss: 0.2200 - val_accuracy: 0.9141 - lr: 0.0010
Epoch 7/50
167/167 [==============================] - 14s 84ms/step - loss: 0.2118 - accuracy: 0.9169 - val_loss: 0.2184 - val_accuracy: 0.9156 - lr: 0.0010
Epoch 8/50
167/167 [==============================] - 13s 79ms/step -

## God Mode Model (does not yield to better results)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras.optimizers import schedules
import numpy as np
# Enable mixed precision training
set_global_policy('mixed_float16')

# Define the model within a strategy scope if using multiple GPUs
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = Sequential([
        Masking(mask_value=0.0, input_shape=(max_len, 2)),  # Mask padding values
        #LSTM(1024, return_sequences=True),  # Increase units
        #Dropout(0.4),
        LSTM(512, return_sequences=True),  # Increase units
        Dropout(0.4),
        LSTM(256, return_sequences=True),
        Dropout(0.4),
        Bidirectional(LSTM(128)),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(1, activation='sigmoid', dtype='float32')  # Ensure output is float32
    ])

    # Define learning rate schedule
    lr_schedule = schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=1000,
        decay_rate=0.9,
        staircase=True)

    # Compile the model with Adam optimizer and learning rate schedule
    optimizer = Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks with correct filepath for model checkpoint
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, save_weights_only=False)

# Train the model with a larger batch size and longer epochs
history = model.fit(X_train, y_train, epochs=100, batch_size=512, validation_data=(X_test, y_test),
                    callbacks=[reduce_lr, model_checkpoint])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
# Load the model from the file
model.save('./drive/MyDrive/NN/NN_god_model.h5')

Epoch 1/100
84/84 [==============================] - 48s 334ms/step - loss: 0.3705 - accuracy: 0.8384 - val_loss: 0.2963 - val_accuracy: 0.8818 - lr: 0.0010
Epoch 2/100
84/84 [==============================] - 17s 205ms/step - loss: 0.2691 - accuracy: 0.8935 - val_loss: 0.3018 - val_accuracy: 0.8740 - lr: 0.0010
Epoch 3/100
84/84 [==============================] - 18s 218ms/step - loss: 0.2542 - accuracy: 0.8998 - val_loss: 0.2421 - val_accuracy: 0.9036 - lr: 0.0010
Epoch 4/100
84/84 [==============================] - 17s 201ms/step - loss: 0.2369 - accuracy: 0.9077 - val_loss: 0.2283 - val_accuracy: 0.9110 - lr: 0.0010
Epoch 5/100
84/84 [==============================] - 18s 215ms/step - loss: 0.2291 - accuracy: 0.9118 - val_loss: 0.2293 - val_accuracy: 0.9110 - lr: 0.0010
Epoch 6/100
84/84 [==============================] - 18s 220ms/step - loss: 0.2226 - accuracy: 0.9129 - val_loss: 0.2311 - val_accuracy: 0.9118 - lr: 0.0010
Epoch 7/100
84/84 [==============================] - 17s 2

KeyboardInterrupt: 